In [1]:
# Basic libraries
import numpy as np
import pandas as pd
import os

# Image processing
import cv2

# Model building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Memory growth set for GPUs.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPUs detected.")

Memory growth set for GPUs.


In [3]:
# Path to the CSV file
data_path = "dataset/fer2013/fer2013/fer2013.csv"  # Update this path to your CSV file

# Load the dataset
df = pd.read_csv(data_path)

# Inspect the first few rows
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
# Convert pixel values from string to numpy array
def preprocess_images(df):
    images = []
    for pixel_sequence in df["pixels"]:
        img_array = np.array(pixel_sequence.split(), dtype="float32")
        img_array = img_array.reshape(48, 48, 1)  # Reshape to 48x48 and single channel
        img_array = img_array / 255.0  # Normalize pixel values to [0, 1]
        images.append(img_array)
    return np.array(images)

# Preprocess images and extract labels
X = preprocess_images(df)
y = df["emotion"].values  # Emotion labels

# Check the shape of images and labels
print("Image data shape:", X.shape)  # Should be (num_samples, 48, 48, 1)
print("Label data shape:", y.shape)  # Should be (num_samples,)

Image data shape: (35887, 48, 48, 1)
Label data shape: (35887,)


In [5]:
# Split into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

print("Training set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Training set shape: (25120, 48, 48, 1)
Validation set shape: (5383, 48, 48, 1)
Test set shape: (5384, 48, 48, 1)


In [6]:
# Define CNN model
model = Sequential([
    Conv2D(64, (3, 3), activation="relu", input_shape=(48, 48, 1), padding="same"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(128, (5, 5), activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Conv2D(512, (3, 3), activation="relu", padding="same"),
    MaxPooling2D((2, 2)),
    Dropout(0.1),
    
    Flatten(),
    Dense(256, activation="relu"),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(512, activation="relu"),
    BatchNormalization(),
    Dropout(0.1),
    
    Dense(7, activation="softmax")
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.005),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       204928    
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 128)      5

In [7]:
# Define callbacks
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("model/emotions.keras", save_best_only=True)

# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/30
785/785 [==============================] - 21s 18ms/step - loss: 1.9944 - accuracy: 0.2299 - val_loss: 1.8428 - val_accuracy: 0.2357
Epoch 2/30
785/785 [==============================] - 14s 17ms/step - loss: 1.6189 - accuracy: 0.3768 - val_loss: 1.7239 - val_accuracy: 0.3572
Epoch 3/30
785/785 [==============================] - 13s 17ms/step - loss: 1.4343 - accuracy: 0.4438 - val_loss: 1.4625 - val_accuracy: 0.4561
Epoch 4/30
785/785 [==============================] - 13s 16ms/step - loss: 1.2911 - accuracy: 0.5031 - val_loss: 1.2777 - val_accuracy: 0.5166
Epoch 5/30
785/785 [==============================] - 13s 16ms/step - loss: 1.2037 - accuracy: 0.5432 - val_loss: 1.2963 - val_accuracy: 0.5008
Epoch 6/30
785/785 [==============================] - 13s 17ms/step - loss: 1.3430 - accuracy: 0.4828 - val_loss: 1.2968 - val_accuracy: 0.5094
Epoch 7/30
785/785 [==============================] - 13s 16ms/step - loss: 1.1803 - accuracy: 0.5537 - val_loss: 1.1728 - val_accuracy:

In [8]:
# Evaluate model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

# Generate classification report
y_pred = np.argmax(model.predict(X_val), axis=1)
print(classification_report(y_val, y_pred))

169/169 [==============================] - 1s 6ms/step - loss: 1.1403 - accuracy: 0.5768
Validation Loss: 1.1403216123580933
Validation Accuracy: 0.5768159031867981
169/169 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.54      0.42      0.47       743
           1       0.65      0.21      0.31        82
           2       0.43      0.27      0.33       768
           3       0.79      0.81      0.80      1348
           4       0.46      0.49      0.48       911
           5       0.76      0.64      0.70       601
           6       0.44      0.69      0.54       930

    accuracy                           0.58      5383
   macro avg       0.58      0.50      0.52      5383
weighted avg       0.58      0.58      0.57      5383



In [11]:
model.save("model/mystery.h5")